In [1]:
import sys
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/")
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/Generative_Model")


sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR/recourse/")



In [2]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch.nn as nn
import pickle as pkl
from numpy import linalg as LA


import ML_Models.data_loader as loader
# from utils import _get_input_subset
# from Recourse_Methods.gradient_methods import SCFE
# from utils import get_recourses, get_performance_measures


%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
## Dataset Prep


from torchvision import  datasets, transforms
from torch.utils.data import DataLoader

# data_name = "compas"
data_name = "adult"
# data_name = "german"
n_starting_instances = 1200
compas_dict = {
        "data_path": '../Data_Sets/COMPAS/',
        "filename_train": 'compas-train.csv',
        "filename_test": 'compas-test.csv',
        "label": "risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }

german_dict = {
        "data_path": '../Data_Sets/German_Credit_Data/',
        "filename_train": 'german-train.csv',
        "filename_test": 'german-test.csv',
        "label": "credit-risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
    }

adult_dict = {
        "data_path": "../Data_Sets/Adult/",
        "filename_train": 'adult-train.csv',
        "filename_test": 'adult-test.csv',
        "label": 'income',
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 25,
        "H2": 25,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }


data_meta_dictionaries = {
        "compas": compas_dict, 
        "adult": adult_dict, 
        "german":german_dict
    }
data_meta_info = data_meta_dictionaries[data_name]


dataset_test = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                 filename=data_meta_info["filename_test"],
                                                 label=data_meta_info["label"])
        
dataset_train = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                  filename=data_meta_info["filename_train"],
                                                  label=data_meta_info["label"])


column_names = pd.read_csv(data_meta_info["data_path"] + data_meta_info["filename_train"]).drop(data_meta_info["label"], axis=1).columns



In [4]:
# Data loader

train_loader = DataLoader(dataset_train, batch_size = 32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size = 32, shuffle=False)

data = [i for i in train_loader]
num_input = len(data[0][0][0])



In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# do a single pass over the data
def epoch(loader, model, opt=None):
    total_loss, total_err = 0.,0.
    for X,y, ind in loader:
        yp = model(X.view(X.shape[0], -1).to(torch.float32))[:,0]
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

model = nn.Linear(num_input, 1)
opt = optim.SGD(model.parameters(), lr=1e-1)
print("Train Err", "Train Loss", "Test Err", "Test Loss", sep="\t")
for i in range(20):
    train_err, train_loss = epoch(train_loader, model, opt)
    test_err, test_loss = epoch(test_loader, model)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")

    

Train Err	Train Loss	Test Err	Test Loss
0.210244	0.428074	0.209176	0.414276
0.193051	0.399845	0.191155	0.400135
0.183210	0.389324	0.190492	0.393516
0.178124	0.382174	0.181537	0.385938
0.175719	0.376910	0.180210	0.381915
0.174669	0.373100	0.178441	0.379023
0.173176	0.370079	0.177336	0.374676
0.173287	0.367649	0.180100	0.373539
0.171850	0.365735	0.177336	0.370519
0.172292	0.363881	0.179878	0.369574
0.171850	0.362759	0.179768	0.368792
0.170827	0.361670	0.177114	0.366602
0.170965	0.360547	0.175456	0.365609
0.170080	0.359508	0.175235	0.367375
0.170937	0.358850	0.174129	0.363749
0.169859	0.358275	0.176783	0.364351
0.170053	0.357782	0.174350	0.362497
0.169859	0.357116	0.175567	0.361979
0.169611	0.356747	0.174903	0.361295
0.169776	0.356260	0.175567	0.360620


In [6]:
epsilon = 0.2
delta = epsilon * model.weight.detach().sign()

# Testing error on adversarial sample 
def epoch_adv(loader, model, delta):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        X = X.to(torch.float32)
#         tmp = ((2*y.float()-1)[:, None]*delta.squeeze(0)).squeeze(0)
        yp = model(X-((2*y.float()-1)[:, None]*delta.squeeze(0)).squeeze(0)).squeeze(-1)
#         print(yp)
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)
print(epoch_adv(test_loader, model, delta[None,None,:,:]))

(0.9354339414040906, 2.774493785244513)


In [7]:
def epoch_robust(loader, model, epsilon, opt=None):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        yp = model(X.view(X.shape[0], -1).to(torch.float32))[:,0] - epsilon*(2*y.float()-1)*model.weight.norm(1)
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)


model_robust = nn.Linear(num_input, 1)
opt = optim.SGD(model_robust.parameters(), lr=1e-1)
epsilon = 0.2
print("Rob. Train Err", "Rob. Train Loss", "Rob. Test Err", "Rob. Test Loss", sep="\t")
for i in range(20):
    train_err, train_loss = epoch_robust(train_loader, model_robust, epsilon, opt)
    test_err, test_loss = epoch_robust(test_loader, model_robust, epsilon)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")    

Rob. Train Err	Rob. Train Loss	Rob. Test Err	Rob. Test Loss
0.250850	0.564319	0.247872	0.558404
0.247837	0.558112	0.247872	0.562316
0.247837	0.558321	0.247872	0.559591
0.247837	0.558190	0.247872	0.557938
0.247837	0.558158	0.247872	0.559580
0.247837	0.558249	0.247872	0.560345
0.247837	0.558258	0.247872	0.560541
0.247837	0.558199	0.247872	0.557459
0.247837	0.557971	0.247872	0.560277
0.247837	0.558430	0.247872	0.557183
0.247837	0.558365	0.247872	0.558690
0.247837	0.558175	0.247872	0.565019
0.247837	0.558203	0.247872	0.559813
0.247837	0.558378	0.247872	0.557233
0.247837	0.558346	0.247872	0.562137
0.247837	0.558364	0.247872	0.559329
0.247837	0.558132	0.247872	0.558841
0.247837	0.558324	0.247872	0.557581
0.247837	0.558274	0.247872	0.560271
0.247837	0.558080	0.247872	0.559178


In [8]:
# Method 3 : Berk's recourse

import lime as lime
import lime.lime_tabular as lime_tabular

import torch
import pandas as pd
import numpy as np
from Recourse_Methods.AR.recourse.cplex_helper import DEFAULT_CPLEX_PARAMETERS
from Recourse_Methods.AR.recourse.flipset import Flipset
from Recourse_Methods.AR.recourse.builder import ActionSet


class AR:
    def __init__(self, classifier, train_data: pd.DataFrame, func_class: str = 'ann', total_items: int = 50):
        self.classifier = classifier
        self.fun_class = func_class
        self.train_data = train_data
        self.total_items = 50

    def _build_lime(self, column_names, discretize_continuous: bool = False, sample_around_instance: bool = True):
        """
        Define a LIME explainer on dataset
        :param data: Dataframe with original train data
        :return: LimeExplainer
        """
        
        # Data preparation
        X = self.train_data.values
        lime_exp = lime.lime_tabular.LimeTabularExplainer(training_data=X,
                                                          discretize_continuous=discretize_continuous,
                                                          sample_around_instance=sample_around_instance, 
                                                          feature_names = column_names)
    
        return lime_exp
        
    def _get_lime_coefficients(self, instance : pd.DataFrame, column_names):
        
        """
        Actionable Recourse is not implemented for non-linear models and non binary categorical data.
        To mitigate the second issue, we have to use LIME to compute coefficients for our Black Box Model.
        :return: List of LIME-Explanations, intercept
        """
        # Prepare instance
        #print(instance.values, type(instance.values), instance.values.reshape((1, -1)), instance.index, instance.index.values)
        # Prepare instance
        inst_to_expl = instance #pd.DataFrame(instance.values.reshape((1, -1)),
                                    #columns=instance.index.values)

        inst_to_expl = instance #pd.DataFrame(instance.numpy().reshape((1, -1)),columns=column_names)

        lime_expl = self._build_lime(column_names)        
        def predict_fn_nn(x):
            return torch.cat((torch.tensor(self.classifier(torch.tensor(x, dtype = torch.float32)) < 0, dtype = torch.float32), 
                            torch.tensor(self.classifier(torch.tensor(x, dtype = torch.float32)) >= 0, dtype = torch.float32)), dim = 1).numpy()  
                             
                             

        explanations = lime_expl.explain_instance(np.squeeze(inst_to_expl.values),
                                                  predict_fn_nn,
                                                  num_features=inst_to_expl.values.shape[1])
#         print("--->>>>", explanations.as_list())
        return explanations.as_list(), explanations.intercept[1]
    

    def generate_counterfactuals(self, column_names, query_instance: pd.DataFrame, target_class: int = 1) -> torch.tensor:
        
        action_set = ActionSet(X=self.train_data)
        
        # Actionable recourse is only defined on linear models
        # To use more complex models, they propose to use local approximation models like LIME
        
        if self.fun_class == 'ann':
            coeff, intercept = self._get_lime_coefficients(query_instance, column_names)
        else:
            coeff, intercept = self.classifier.get_coefficients()
            coeff = coeff[1].detach().numpy().tolist()
            intercept = intercept[1].detach().numpy().tolist()
        
        # Match LIME Coefficients with actionable recourse data
        # if LIME coef. is in ac_columns then use coefficient else 0
        ac_columns = self.train_data.columns
        rest_columns = [x for x in column_names if x not in ac_columns]
        
        # Turn top 10 LIME coefficients into list with coefficients containing all features
        # Features coefficients which are not in the top 10 are set to 0
        if self.fun_class == 'ann':
            coefficients = np.zeros(ac_columns.shape)
            for i, feature in enumerate(ac_columns):
                for t in coeff:
                    if t[0].find(feature) != -1:
                        coefficients[i] += t[1]
        else:
            coefficients = coeff
            
        # Align action set to coefficients
        action_set.set_alignment(coefficients=coefficients)
        
        # Build counterfactuals
        rest_df = query_instance[rest_columns].values.reshape((1, -1))
        rest_df = pd.DataFrame(rest_df, columns=rest_columns)
        inst_for_ac = query_instance[ac_columns].values.reshape((1, -1))
        inst_for_ac = pd.DataFrame(inst_for_ac, columns=ac_columns)
        
        fb = Flipset(
            x=inst_for_ac.values,
            action_set=action_set,
            mip_cost_type='l2',
            coefficients=coefficients,
            intercept=intercept
        )
        
        # Fit AC and build counterfactual
        fb_set = fb.populate(enumeration_type='distinct_subsets', total_items=self.total_items)
        actions_flipset = fb_set.actions
        actions_flipset = sorted(actions_flipset, key=lambda x: np.sqrt(np.dot(np.array(x), np.array(x))))
        last_object = len(actions_flipset) - 1
        for idx, action in enumerate(actions_flipset):
            counterfactual = inst_for_ac.values + action
            counterfactual = pd.DataFrame(counterfactual, columns=ac_columns)
            counterfactual[rest_columns] = rest_df[rest_columns]
            counterfactual = counterfactual[
                query_instance.columns]  # Arrange instance and counterfactual in same column order
                
        distance_l1 = LA.norm(counterfactual.values - query_instance.values, 1)
        distance_l2 = LA.norm(counterfactual.values - query_instance.values, 2)
        
        return counterfactual, distance_l1, distance_l2

    
    


In [9]:
def _get_input_subset(model_sub, inputs: torch.tensor,
                      subset_size: int = 100,
                      decision_threshold: float = 0) -> torch.tensor:
    
    """
    Get negatively classified inputs & return their predictions
    """
    
    yhat = (model_sub(inputs) > decision_threshold) * 1
    check = (model_sub(inputs) < decision_threshold).detach().numpy()
    selected_indices = np.where(check)[0]
    input_subset = inputs[selected_indices]
    predicted_label_subset = yhat[selected_indices]
    return input_subset[0:subset_size, :], predicted_label_subset[0:subset_size] #[0:subset_size]

In [10]:
# This method makes the "Manifold assumption" and uses random search in latent space

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
subset_size = 100

        
testloader = DataLoader(dataset_test,
                        batch_size=data_meta_info["n_starting_instances"],
                        shuffle=False)

data_iter = iter(testloader)
inputs, labels, indeces = data_iter.next()
inputs = inputs.to(device).float()

inputs, predicted_classes = _get_input_subset(model, inputs, subset_size=subset_size, decision_threshold=0)
inputs = inputs.numpy()
inps_base = pd.DataFrame(inputs)
inps_base.columns = column_names

ar = AR(classifier=model, train_data=inps_base)


testloader = DataLoader(dataset_test,
                        batch_size=data_meta_info["n_starting_instances"],
                        shuffle=False)

data_iter = iter(testloader)
inputs, labels, indeces = data_iter.next()
inputs = inputs.to(device).float()

inputs, predicted_classes = _get_input_subset(model_robust, inputs, subset_size=subset_size, decision_threshold=0)
inputs = inputs.numpy()
inps_ro = pd.DataFrame(inputs)
inps_ro.columns = column_names


ar_r = AR(classifier=model_robust, train_data=inps_ro)

In [12]:
# Generate counterfactuals for  

# inputs = pd.DataFrame(inputs, columns = column_names)

# counterfactual, distance_l1, distance_l2 = ar.generate_counterfactuals(
#                     column_names, query_instance= inputs.loc[[0]], target_class=1)


# print(inps_ro.shape) #, model_robust(inps_ro.loc[[0]]))


distances_base_ar = [ar.generate_counterfactuals(
                    column_names, query_instance= inps_base.loc[[i]], target_class=1) for i in range(1)] #range(inps_base.shape[0])]


distances_ro_ar = [ar_r.generate_counterfactuals(
                    column_names, query_instance= inps_ro.loc[[i]], target_class=1) for i in range(1)] # range(inps_base.shape[0])]





# print(counterfactual, distance_l1, distance_l2)


flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
flip_direction -1.0
flip_direction -1.0
flip_direction 1.0
flip_direction 1.0
flip_direction 1.0
In MIP Build..
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   -1 0
--->>> compatible  True
-->   -1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
-->   1 0
--->>> compatible  True
Output:  {'age': (array([0.        , 0.00631233, 0.01275891, 0.01920548, 0.02565206,
       0.03209863, 0.03854521, 0.04499178, 0.05143836, 0.05788493,
       0.06433151, 0.07077808, 0.07722466, 0.08367123, 0.09011781,
       0.09656439, 0.10301096, 0.10945754, 0.11590411, 0.12235069,
      

/var/folders/kp/0wxv5tl156vgbnlj8jqt9b7m0000gn/T/ipykernel_49103/1585168897.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat((torch.tensor(self.classifier(torch.tensor(x, dtype = torch.float32)) < 0, dtype = torch.float32),
/var/folders/kp/0wxv5tl156vgbnlj8jqt9b7m0000gn/T/ipykernel_49103/1585168897.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.classifier(torch.tensor(x, dtype = torch.float32)) >= 0, dtype = torch.float32)), dim = 1).numpy()


obtained 50 items in 1.2 seconds
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
flip_direction 0.0
In MIP Build..
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
-->   0 0
--->>> compatible  False
Output:  {}
->Actions :  {}


/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR/recourse/flipset.py:395: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._df = self._df.append(row_data, ignore_index = True, sort = True)[self._df.columns.tolist()]
/var/folders/kp/0wxv5tl156vgbnlj8jqt9b7m0000gn/T/ipykernel_49103/1585168897.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat((torch.tensor(self.classifier(torch.tensor(x, dtype = torch.float32)) < 0, dtype = torch.float32),
/var/folders/kp/0wxv5tl156vgbnlj8jqt9b7m0000gn/T/ipykernel_49103/1585168897.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch

KeyError: 'cost_var_names'